In [1]:
import zipfile
import os
import json

# Function to handle the PBIP folder and extract report.json
def extract_report_and_model(zip_file):
    # Get the base name of the uploaded zip file, excluding the extension
    inner_folder_name = os.path.splitext(zip_file.name)[0] 
    # Extract the zip file
    extract_path = '\mnt\data\pbip_extracted'
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    
    inner_folder_path = os.path.join(extract_path, inner_folder_name)
    # Print the contents of the inner folder
    inner_folder_contents = os.listdir(inner_folder_path)
    report_folder_path = None
    model_bim_path = None
    
    # Look for the folder that ends with '.Report' or '.SemanticModel'
    for folder in inner_folder_contents:
        full_folder_path = os.path.join(inner_folder_path, folder)
        if folder.endswith('.Report') and os.path.isdir(full_folder_path):
            report_folder_path = full_folder_path
        elif folder.endswith('.SemanticModel') and os.path.isdir(full_folder_path):
            model_folder_path = full_folder_path
            print(model_folder_path)

    # Extract report.json
    if report_folder_path:
        report_json_path = os.path.join(report_folder_path, 'report.json')
        print(report_json_path)
        with open(report_json_path, 'r', encoding='utf-8') as file:
            report_json_content = json.load(file)
    else:
        report_json_content = None

    model_bim_content = None

    # Extract model.bim
    if model_folder_path:
        model_bim_path = os.path.join(model_folder_path, 'model.bim')
        print("model bim path is: ", model_bim_path)
        # if os.path.exists(model_bim_path):
        #     print("model bim path exists")
        with open(model_bim_path, 'r', encoding='utf-8') as file:
            model_bim_content = json.load(file)
    else:
        model_bim_content = None

    return report_json_content, model_bim_content, inner_folder_path, report_json_path, model_bim_path

<>:10: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18348\1238016480.py:10: SyntaxWarning: invalid escape sequence '\m'
  extract_path = '\mnt\data\pbip_extracted'


In [2]:
# Example usage
zip_file_path = 'inchanel global.zip' 
zip_file = open(zip_file_path, 'rb')  # This line opens the zip file
report_json, model_bim, folder_path, report_path, model_path = extract_report_and_model(zip_file)

\mnt\data\pbip_extracted\inchanel global\inchanel global.SemanticModel
\mnt\data\pbip_extracted\inchanel global\inchanel global.Report\report.json
model bim path is:  \mnt\data\pbip_extracted\inchanel global\inchanel global.SemanticModel\model.bim


In [3]:
# report_json

In [4]:
report_json['sections'][6].keys()

dict_keys(['config', 'displayName', 'displayOption', 'filters', 'height', 'name', 'ordinal', 'visualContainers', 'width'])

In [5]:
visuals = []
for i in range(len(report_json['sections'][6]['visualContainers'])):
    config_data = report_json['sections'][6]['visualContainers'][i].get("config", {})
    config_data = json.loads(config_data)
    visual_type_to_be_excluded = ['actionButton', 'image', 'shape', 'textbox', '']
    if config_data.get("singleVisual", {}).get("visualType", "") not in visual_type_to_be_excluded:
        visuals.append(config_data.get("singleVisual", {}).get("visualType", ""))
        # print(i, config_data.get("singleVisual", {}).get("visualType", ""))

In [48]:
len(visuals)
distinct_visuals = set(visuals)
# distinct_visuals
# visuals

In [7]:
test_visual_config = report_json['sections'][6]['visualContainers'][9]['config']
test_visual_config = json.loads(test_visual_config)

In [8]:
print(test_visual_config.keys())
# test_visual_config
print("the attributes of a single visual are:", test_visual_config.get("singleVisual", {}).keys())

dict_keys(['name', 'layouts', 'singleVisual', 'parentGroupName'])
the attributes of a single visual are: dict_keys(['visualType', 'projections', 'prototypeQuery', 'drillFilterOtherVisuals', 'objects', 'vcObjects'])


In [9]:
test_visual_config.get("singleVisual", {}).get("prototypeQuery", {})

{'Version': 2,
 'From': [{'Name': 'd', 'Entity': 'Date', 'Type': 0},
  {'Name': '_', 'Entity': '_KPI_Customer', 'Type': 0}],
 'Select': [{'Column': {'Expression': {'SourceRef': {'Source': 'd'}},
    'Property': 'MonthYear'},
   'Name': 'Date.MonthYear',
   'NativeReferenceName': 'MonthYear'},
  {'Measure': {'Expression': {'SourceRef': {'Source': '_'}},
    'Property': 'Onboarded'},
   'Name': '_KPI_Customer.Onboarded',
   'NativeReferenceName': 'Onboarded'}]}

In [10]:
test_visual_config.get("singleVisual", {}).get("columnProperties", {})

{}

In [11]:
test_visual_config.get("vcObjects", {}).get("title", [])

[]

In [22]:
def extract_relevant_elements_dashboard_summary(json_data):
    sections_list = []

    for section in json_data.get("sections", []):
        # only include the pages which are not hidden in the dashboard
        page_config = json.loads(section['config'])
        if page_config.get('visibility') != 1:
            # Store displayName to know the section/page name
            section_summary = {
                "displayName": section.get("displayName", ""),
                "filters": section.get("filters", ""),
                "ordinal": section.get("ordinal", ""),
                "visualContainers": []
            }

            # Process each visual container in the section
            for visual in section.get("visualContainers", []):
                # Parse config if it's a string
                config_data = visual.get("config", {})
                if isinstance(config_data, str):
                    config_data = json.loads(config_data)

                    visual_type_to_be_excluded = ['actionButton', 'image', 'shape', 'textbox', '']
                    if config_data.get("singleVisual", {}).get("visualType", "") not in visual_type_to_be_excluded:
                        # Extract relevant visual properties
                        visual_summary = {
                            "visualType": config_data.get("singleVisual", {}).get("visualType", ""),
                            "projections": config_data.get("singleVisual", {}).get("projections", []),
                            "prototypeQuery": config_data.get("singleVisual", {}).get("prototypeQuery", {}),
                            "title": config_data.get("vcObjects", {}).get("title", []),
                            "filters": visual.get("filters", [])
                        }

                        # Add visual summary if it has useful data
                        if any(visual_summary.values()):
                            section_summary["visualContainers"].append(visual_summary)

            # Add the section's displayName and extracted_data to the list
            sections_list.append({
                "displayName": section.get("displayName", ""),
                "extracted_data": section_summary
            })

    return sections_list


In [23]:
extracted_report = extract_relevant_elements_dashboard_summary(report_json)

In [26]:
extracted_report[1]

{'displayName': 'Scope FA',
 'extracted_data': {'displayName': 'Scope FA',
  'filters': '[]',
  'ordinal': 4,
  'visualContainers': [{'visualType': 'card',
    'projections': {'Values': [{'queryRef': 'Mesure.total_nb_of_chats'}]},
    'prototypeQuery': {'Version': 2,
     'From': [{'Name': 'm', 'Entity': 'Mesure', 'Type': 0}],
     'Select': [{'Measure': {'Expression': {'SourceRef': {'Source': 'm'}},
        'Property': 'Total of Chats'},
       'Name': 'Mesure.total_nb_of_chats',
       'NativeReferenceName': 'Total of Chats'}],
     'OrderBy': [{'Direction': 2,
       'Expression': {'Measure': {'Expression': {'SourceRef': {'Source': 'm'}},
         'Property': 'Total of Chats'}}}]},
    'title': [],
    'filters': '[]'},
   {'visualType': 'card',
    'projections': {'Values': [{'queryRef': 'Mesure.Nb Chat sent By FAs'}]},
    'prototypeQuery': {'Version': 2,
     'From': [{'Name': 'm', 'Entity': 'Mesure', 'Type': 0}],
     'Select': [{'Measure': {'Expression': {'SourceRef': {'Source'

In [16]:
report_json.keys()

dict_keys(['config', 'layoutOptimization', 'resourcePackages', 'sections'])

In [17]:
report_json['sections'][6].keys()

dict_keys(['config', 'displayName', 'displayOption', 'filters', 'height', 'name', 'ordinal', 'visualContainers', 'width'])

In [18]:
# Initialize a set to store distinct visual types
distinct_visual_types = set()

for i in range(len(report_json['sections'][6]['visualContainers'])):
    visuel = report_json['sections'][6]['visualContainers'][i]['config']
    parsed_data = json.loads(visuel)  # Parse the JSON string
    visual_type = parsed_data.get("singleVisual", {}).get("visualType")
    
    if visual_type:  # Check if visual_type is not None
        distinct_visual_types.add(visual_type)  # Add to the set

# Print the distinct visual types
print("Distinct Visual Types:", distinct_visual_types)

Distinct Visual Types: {'textbox', 'image', 'clusteredColumnChart', 'donutChart', 'slicer', 'actionButton', 'card', 'shape'}


In [19]:
visuel_string = report_json['sections'][6]['visualContainers'][5]['config']
parsed_data = json.loads(visuel_string)
parsed_data.keys()

dict_keys(['name', 'layouts', 'singleVisualGroup'])

In [ ]:
visual = parsed_data['singleVisual']
print(visual)

# Remove the 'objects' attribute
if 'objects' in visual.keys():
    print(True)
    del visual['objects']

{'visualType': 'card', 'projections': {'Values': [{'queryRef': 'Mesure.INCHANEL Total Users (Since Start)'}]}, 'prototypeQuery': {'Version': 2, 'From': [{'Name': 'm', 'Entity': 'Mesure', 'Type': 0}], 'Select': [{'Measure': {'Expression': {'SourceRef': {'Source': 'm'}}, 'Property': 'INCHANEL ACTIVE Users'}, 'Name': 'Mesure.INCHANEL Total Users (Since Start)', 'NativeReferenceName': 'Active Clients on the period'}], 'OrderBy': [{'Direction': 2, 'Expression': {'Measure': {'Expression': {'SourceRef': {'Source': 'm'}}, 'Property': 'INCHANEL ACTIVE Users'}}}]}, 'columnProperties': {'Mesure.INCHANEL Total Users (Since Start)': {'displayName': 'Active Clients on the period'}}, 'drillFilterOtherVisuals': True, 'hasDefaultSort': True, 'objects': {'labels': [{'properties': {'color': {'solid': {'color': {'expr': {'ThemeDataColor': {'ColorId': 1, 'Percent': 0}}}}}, 'fontSize': {'expr': {'Literal': {'Value': '25D'}}}, 'bold': {'expr': {'Literal': {'Value': 'true'}}}}}], 'categoryLabels': [{'properti

In [ ]:
visual.keys()

dict_keys(['visualType', 'projections', 'prototypeQuery', 'columnProperties', 'drillFilterOtherVisuals', 'hasDefaultSort', 'vcObjects'])

In [ ]:


# Extract the visualType
visual_type = parsed_data.get("singleVisual", {}).get("visualType")

print("Visual Type:", visual_type)

Visual Type: image


In [ ]:
# with open('data.json', 'w') as json_file:
#     json.dump(model_bim, json_file, indent=4) 

### Dataset

In [3]:
model_bim

{'compatibilityLevel': 1567,
 'model': {'annotations': [{'name': 'PBI_QueryOrder',
    'value': '["fact_wishlist","dim_calendar","dim_boutique","dim_client","Mesure","fact_mirror","fact_suggestion_board","fact_message","server_id","database_id","storage_id"]'},
   {'name': '__PBI_TimeIntelligenceEnabled', 'value': '1'},
   {'name': 'PBIDesktopVersion', 'value': '2.138.782.0 (24.11)'},
   {'name': 'PBI_ProTooling', 'value': '["DaxQueryView_Desktop","DevMode"]'}],
  'culture': 'fr-FR',
  'cultures': [{'name': 'fr-FR',
    'linguisticMetadata': {'content': {'Entities': {'dim_boutique': {'Definition': {'Binding': {'ConceptualEntity': 'dim_boutique'}},
        'State': 'Generated',
        'Terms': [{'dim boutique': {'State': 'Generated'}},
         {'dim_boutique': {'State': 'Generated',
           'Type': 'Noun',
           'Weight': 0.99}},
         {'boutique': {'State': 'Generated', 'Weight': 0.97}}]},
       'dim_boutique.UTC_closing_hour_summer': {'Definition': {'Binding': {'Conceptu

In [23]:
model_bim['model'].keys()

dict_keys(['annotations', 'culture', 'cultures', 'dataAccessOptions', 'defaultPowerBIDataSourceVersion', 'expressions', 'queryGroups', 'relationships', 'sourceQueryCulture', 'tables'])

In [51]:
model_bim['model']['expressions']

[{'name': 'server_id',
  'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
   {'name': 'PBI_ResultType', 'value': 'Text'}],
  'description': 'Azure server name',
  'expression': '"asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
  'kind': 'm',
  'lineageTag': 'acf363bb-3a3b-4c63-8ce9-64df713220f6',
  'queryGroup': 'parameters'},
 {'name': 'database_id',
  'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
   {'name': 'PBI_ResultType', 'value': 'Text'}],
  'expression': '"dhfsh-gold" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
  'kind': 'm',
  'lineageTag': 'a29a4a2c-53d5-46aa-a171-21ed366054bd',
  'queryGroup': 'parameters'},
 {'name': 'storage_id',
  'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
   {'name': 'PBI_ResultType', 'value': 'Text'}],
  'description': 'Azure blob storage id',
  'expression':

In [ ]:
model_bim['model']['relationships']

[{'name': 'f14ab861-6de1-4ee5-83f7-0d54e5b5938c',
  'fromColumn': 'Date',
  'fromTable': 'dim_calendar',
  'joinOnDateBehavior': 'datePartOnly',
  'toColumn': 'Date',
  'toTable': 'LocalDateTable_822da7e9-510e-4037-be69-095aaa0a2010'},
 {'name': 'c9107009-412b-4574-980c-7ed8c5a62a86',
  'fromColumn': 'Start of the Month',
  'fromTable': 'dim_calendar',
  'joinOnDateBehavior': 'datePartOnly',
  'toColumn': 'Date',
  'toTable': 'LocalDateTable_4de7692d-409c-4065-9503-d0b84e2f7489'},
 {'name': '3b1de498-ce49-4932-8bda-832bd1565be8',
  'fromColumn': 'End of the Month',
  'fromTable': 'dim_calendar',
  'joinOnDateBehavior': 'datePartOnly',
  'toColumn': 'Date',
  'toTable': 'LocalDateTable_9a1422c1-22fb-462c-9059-dcf138aec3e7'},
 {'name': '24216199-4312-4a98-852f-b1a9e44c6405',
  'fromColumn': 'Start of the Week',
  'fromTable': 'dim_calendar',
  'joinOnDateBehavior': 'datePartOnly',
  'toColumn': 'Date',
  'toTable': 'LocalDateTable_fdf3cfa6-ac4f-4279-80cf-cd75cf0f5ed8'},
 {'name': '674c40

In [52]:
# Define the keywords to exclude
excluded_keywords = ["LocalDateTable", "DateTableTemplate"]

included_table = []
for i in range(len(model_bim['model']['tables'])):
    # Check if the name does not contain any excluded keyword
    if not any(keyword in model_bim['model']['tables'][i]['name'] for keyword in excluded_keywords):
        included_table.append(model_bim['model']['tables'][i])
        # Print the name of the included table
        print(model_bim['model']['tables'][i]['name'])


fact_wishlist
dim_calendar
Mesure
fact_mirror
fact_suggestion_board
fact_message
dim_client
dim_boutique


In [53]:
len(included_table)

8

In [33]:
model_bim['model']['tables'][8]['partitions']

[{'name': 'fact_mirror',
  'mode': 'import',
  'queryGroup': 'fact',
  'source': {'expression': ['let',
    '    Source = Sql.Database(',
    '        "asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net", ',
    '        database_id, ',
    '        [Query = "SELECT *',
    '                  FROM OPENROWSET(',
    '                        BULK \'https://" & storage_id & ".dfs.core.windows.net/digital-platine/fact/fact_mirror/\',',
    "                        FORMAT = 'DELTA'",
    '                    ) AS [result]"',
    '        ]',
    '    )',
    'in',
    '    Source'],
   'type': 'm'}}]

In [43]:
model_bim['model']['tables'][7].keys()

dict_keys(['name', 'annotations', 'lineageTag', 'measures', 'partitions'])

In [34]:
model = model_bim['model']

# Define the keywords to exclude
excluded_keywords = ["LocalDateTable", "DateTableTemplate"]
all_measures =[]
# Filter tables to exclude ones containing the excluded keywords
for table in model.get("tables", []):
    if not any(keyword in table['name'] for keyword in excluded_keywords):
        print(table['name'])
        print(table['partitions'])


fact_wishlist
[{'name': 'fact_wishlist', 'mode': 'import', 'queryGroup': 'fact', 'source': {'expression': ['let', '    Source = Sql.Database(', '        server_id, ', '        database_id, ', '        [Query = "SELECT *', '                  FROM OPENROWSET(', '                        BULK \'https://" & storage_id & ".dfs.core.windows.net/digital-platine/fact/fact_wishlist/\',', "                        FORMAT = 'DELTA'", '                    ) AS [result]"', '        ]', '    ),', '    #"Modify Date Column Format" = Table.TransformColumnTypes(Source,{{"lab_updatestatus", type date}}),', '    #"Lignes filtrées" = Table.SelectRows(#"Modify Date Column Format", each true)', 'in', '    #"Lignes filtrées"'], 'type': 'm'}}]
dim_calendar
[{'name': 'dim_calendar', 'mode': 'import', 'queryGroup': 'dimension', 'source': {'expression': ['let', '    Source = let', '    Source = let', '    Source = "",', '    StartDate = #date(2019, 1, 1),', '    EndDate = #date(2029, 1, 1),', '    Duration = Durat

In [29]:
model = model_bim['model']

# Define the keywords to exclude
excluded_keywords = ["LocalDateTable", "DateTableTemplate"]
all_measures =[]
# Filter tables to exclude ones containing the excluded keywords
for table in model.get("tables", []):
    if not any(keyword in table['name'] for keyword in excluded_keywords):
        if "measures" in table.keys():
            print("the table {} contains measures".format(table['name']))
            for measure in table['measures']:
                all_measures.append(measure)

all_measures


the table Mesure contains measures


[{'name': 'Nb of Clients who used the Wishlist',
  'displayFolder': 'wishlist',
  'expression': ['', 'DISTINCTCOUNT(fact_wishlist[fky_client])', ''],
  'formatString': '0',
  'lineageTag': 'b04888e0-5770-4b38-be96-d2fc9d2b53c2'},
 {'name': 'Wishlist usage frequency per client',
  'annotations': [{'name': 'PBI_FormatHint',
    'value': '{"isGeneralNumber":true}'}],
  'displayFolder': 'wishlist',
  'expression': ['',
   'VAR distinct_days = ',
   "DISTINCTCOUNT( 'fact_wishlist'[lab_updatestatus] )",
   '',
   'VAR t =',
   '    SUMMARIZE(',
   '        fact_wishlist,',
   '        fact_wishlist[fky_client],',
   '        "val", distinct_days',
   '    )',
   '',
   'RETURN',
   '    AVERAGEX(',
   '        t,',
   '        [val]',
   '    )',
   ''],
  'lineageTag': '1117dc79-4a80-4df0-b02f-7ffd4be90d35'},
 {'name': 'Nb of Mirror Sessions',
  'displayFolder': 'mirror',
  'expression': ['',
   'CALCULATE(',
   '    COUNTROWS(fact_mirror),',
   '    FILTER(',
   '        fact_mirror,',
   

In [8]:
model_bim['model']['tables'][8]['partitions']

[{'name': 'fact_mirror',
  'mode': 'import',
  'queryGroup': 'fact',
  'source': {'expression': ['let',
    '    Source = Sql.Database(',
    '        "asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net", ',
    '        database_id, ',
    '        [Query = "SELECT *',
    '                  FROM OPENROWSET(',
    '                        BULK \'https://" & storage_id & ".dfs.core.windows.net/digital-platine/fact/fact_mirror/\',',
    "                        FORMAT = 'DELTA'",
    '                    ) AS [result]"',
    '        ]',
    '    )',
    'in',
    '    Source'],
   'type': 'm'}}]

In [7]:
def extract_relevant_elements_dashboard_summary(json_dataset):
    model = json_dataset.get('model', {})
    # Initialize the extracted dataset
    extracted_json_dataset = {       
        "tables": {
            "expressions": model.get("expressions", []),
            "table_partitions": []
        },
        "measures": []
    }

    # Define the keywords to exclude
    excluded_keywords = ["LocalDateTable", "DateTableTemplate"]

    # Filter tables to exclude ones containing the excluded keywords
    for table in model.get("tables", []):
        if not any(keyword in table.get('name', '') for keyword in excluded_keywords):
            # Add table partitions if they exist
            partitions = table.get('partitions', [])
            if partitions:
                extracted_json_dataset["tables"]["table_partitions"].extend(partitions)

            # Add measures if they exist
            if "measures" in table:
                print(f"The table '{table['name']}' contains measures.")
                extracted_json_dataset["measures"].extend(table["measures"])
    
    return extracted_json_dataset


In [8]:
test_extraction = extract_relevant_elements_dashboard_summary(model_bim)
test_extraction

The table 'Mesure' contains measures.


{'tables': {'expressions': [{'name': 'server_id',
    'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
     {'name': 'PBI_ResultType', 'value': 'Text'}],
    'description': 'Azure server name',
    'expression': '"asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
    'kind': 'm',
    'lineageTag': 'acf363bb-3a3b-4c63-8ce9-64df713220f6',
    'queryGroup': 'parameters'},
   {'name': 'database_id',
    'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
     {'name': 'PBI_ResultType', 'value': 'Text'}],
    'expression': '"dhfsh-gold" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
    'kind': 'm',
    'lineageTag': 'a29a4a2c-53d5-46aa-a171-21ed366054bd',
    'queryGroup': 'parameters'},
   {'name': 'storage_id',
    'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
     {'name': 'PBI_ResultType', 'value': 'Text'

In [12]:
data_table = test_extraction['tables']

In [17]:
data_table

{'expressions': [{'name': 'server_id',
   'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
    {'name': 'PBI_ResultType', 'value': 'Text'}],
   'description': 'Azure server name',
   'expression': '"asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
   'kind': 'm',
   'lineageTag': 'acf363bb-3a3b-4c63-8ce9-64df713220f6',
   'queryGroup': 'parameters'},
  {'name': 'database_id',
   'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
    {'name': 'PBI_ResultType', 'value': 'Text'}],
   'expression': '"dhfsh-gold" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
   'kind': 'm',
   'lineageTag': 'a29a4a2c-53d5-46aa-a171-21ed366054bd',
   'queryGroup': 'parameters'},
  {'name': 'storage_id',
   'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
    {'name': 'PBI_ResultType', 'value': 'Text'}],
   'description': 'Azure

In [10]:
data = test_extraction['measures']

In [11]:
def extract_name_and_expression(elements):
    result = []
    for element in elements:
        name = element.get('name')
        # Combine the non-empty parts of the 'expression' list into a single string
        expression = " ".join(part.strip() for part in element.get('expression', []) if part.strip())
        result.append({'name': name, 'expression': expression})
    return result

result = extract_name_and_expression(data)
for item in result:
    print(item)


{'name': 'Nb of Clients who used the Wishlist', 'expression': 'DISTINCTCOUNT(fact_wishlist[fky_client])'}
{'name': 'Wishlist usage frequency per client', 'expression': 'VAR distinct_days = DISTINCTCOUNT( \'fact_wishlist\'[lab_updatestatus] ) VAR t = SUMMARIZE( fact_wishlist, fact_wishlist[fky_client], "val", distinct_days ) RETURN AVERAGEX( t, [val] )'}
{'name': 'Nb of Mirror Sessions', 'expression': 'CALCULATE( COUNTROWS(fact_mirror), FILTER( fact_mirror, fact_mirror[is_session_starting_time_validated] = TRUE() && fact_mirror[experience_Duration_valid] = TRUE() && fact_mirror[lab_sessiontype] = "external" ) )'}
{'name': 'Share of Mirror Sessions attached to a client', 'expression': 'DIVIDE( CALCULATE( [Nb of Mirror Sessions] , fact_mirror[connection_mode] = "Client" ) , [Nb of Mirror Sessions] )'}
{'name': 'Nb of clients attached to Mirror Sessions', 'expression': 'CALCULATE( DISTINCTCOUNT(fact_mirror[fky_client]), FILTER( fact_mirror, fact_mirror[is_session_starting_time_validated] =

In [39]:
test_extraction.keys()

dict_keys(['expressions', 'tables'])

In [12]:
model_bim['model']['tables'][7]

{'name': 'Mesure',
 'annotations': [{'name': 'PBI_ResultType', 'value': 'Table'}],
 'lineageTag': 'f6b28902-600c-4245-9ec3-14d242fe77ea',
 'measures': [{'name': 'Nb of Clients who used the Wishlist',
   'displayFolder': 'wishlist',
   'expression': ['', 'DISTINCTCOUNT(fact_wishlist[fky_client])', ''],
   'formatString': '0',
   'lineageTag': 'b04888e0-5770-4b38-be96-d2fc9d2b53c2'},
  {'name': 'Wishlist usage frequency per client',
   'annotations': [{'name': 'PBI_FormatHint',
     'value': '{"isGeneralNumber":true}'}],
   'displayFolder': 'wishlist',
   'expression': ['',
    'VAR distinct_days = ',
    "DISTINCTCOUNT( 'fact_wishlist'[lab_updatestatus] )",
    '',
    'VAR t =',
    '    SUMMARIZE(',
    '        fact_wishlist,',
    '        fact_wishlist[fky_client],',
    '        "val", distinct_days',
    '    )',
    '',
    'RETURN',
    '    AVERAGEX(',
    '        t,',
    '        [val]',
    '    )',
    ''],
   'lineageTag': '1117dc79-4a80-4df0-b02f-7ffd4be90d35'},
  {'nam

In [4]:
model_bim['model']['tables'][7]['measures']

[{'name': 'Nb of Clients who used the Wishlist',
  'displayFolder': 'wishlist',
  'expression': ['', 'DISTINCTCOUNT(fact_wishlist[fky_client])', ''],
  'formatString': '0',
  'lineageTag': 'b04888e0-5770-4b38-be96-d2fc9d2b53c2'},
 {'name': 'Wishlist usage frequency per client',
  'annotations': [{'name': 'PBI_FormatHint',
    'value': '{"isGeneralNumber":true}'}],
  'displayFolder': 'wishlist',
  'expression': ['',
   'VAR distinct_days = ',
   "DISTINCTCOUNT( 'fact_wishlist'[lab_updatestatus] )",
   '',
   'VAR t =',
   '    SUMMARIZE(',
   '        fact_wishlist,',
   '        fact_wishlist[fky_client],',
   '        "val", distinct_days',
   '    )',
   '',
   'RETURN',
   '    AVERAGEX(',
   '        t,',
   '        [val]',
   '    )',
   ''],
  'lineageTag': '1117dc79-4a80-4df0-b02f-7ffd4be90d35'},
 {'name': 'Nb of Mirror Sessions',
  'displayFolder': 'mirror',
  'expression': ['',
   'CALCULATE(',
   '    COUNTROWS(fact_mirror),',
   '    FILTER(',
   '        fact_mirror,',
   

In [ ]:
model_bim['model']['tables'][0].keys()

dict_keys(['name', 'annotations', 'columns', 'hierarchies', 'isHidden', 'isPrivate', 'lineageTag', 'partitions'])

In [67]:
dictionary_table = model_bim['model']['tables'][0]
dictionary_table.remove['annotations']

[{'name': 'PBI_ResultType', 'value': 'Table'},
 {'name': 'PBI_NavigationStepName', 'value': 'Navigation'}]

In [10]:
relevant_info = extract_relevant_elements_dashboard_summary(model_bim)

In [11]:
relevant_info

{'expressions': [{'name': 'server_id',
   'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
    {'name': 'PBI_ResultType', 'value': 'Text'}],
   'description': 'Azure server name',
   'expression': '"asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
   'kind': 'm',
   'lineageTag': 'acf363bb-3a3b-4c63-8ce9-64df713220f6',
   'queryGroup': 'parameters'},
  {'name': 'database_id',
   'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
    {'name': 'PBI_ResultType', 'value': 'Text'}],
   'expression': '"dhfsh-gold" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
   'kind': 'm',
   'lineageTag': 'a29a4a2c-53d5-46aa-a171-21ed366054bd',
   'queryGroup': 'parameters'},
  {'name': 'storage_id',
   'annotations': [{'name': 'PBI_NavigationStepName', 'value': 'Navigation'},
    {'name': 'PBI_ResultType', 'value': 'Text'}],
   'description': 'Azure

In [ ]:
import json

def extract_relevant_parts(data):
    # Initialize a dictionary to hold extracted information
    relevant_parts = {
        "DataSources": [],
        "DAXCalculations": [],
        "Relationships": []
    }

    # Extract data sources
    if "expressions" in data.get("model", {}):
        for expression in data["model"]["expressions"]:
            relevant_parts["DataSources"].append({
                "Name": expression.get("name"),
                "Expression": expression.get("expression"),
                "QueryGroup": expression.get("queryGroup")
            })

    # Extract DAX calculations (calculated columns and measures)
    if "tables" in data.get("model", {}):
        for table in data["model"]["tables"]:
            # for column in table.get("columns", []):
            #     if column.get("type") == "calculated":
            #         relevant_parts["DAXCalculations"].append({
            #             "Table": table.get("name"),
            #             "Column": column.get("name"),
            #             "Expression": column.get("expression")
            #         })
            for measure in table.get("measures", []):
                relevant_parts["DAXCalculations"].append({
                    "Table": table.get("name"),
                    "Measure": measure.get("name"),
                    "Expression": measure.get("expression")
                })

    # Extract relationships between tables
    if "relationships" in data.get("model", {}):
        for relationship in data["model"]["relationships"]:
            relevant_parts["Relationships"].append({
                "FromTable": relationship.get("fromTable"),
                "FromColumn": relationship.get("fromColumn"),
                "ToTable": relationship.get("toTable"),
                "ToColumn": relationship.get("toColumn"),
                "JoinBehavior": relationship.get("joinOnDateBehavior", "standard")
            })

    return relevant_parts


In [ ]:
relevant_info = extract_relevant_parts(model_bim)

# Display the results
relevant_info

{'DataSources': [{'Name': 'server_id',
   'Expression': '"asa-dthglbfsh-weu-uat-ondemand.sql.azuresynapse.net" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
   'QueryGroup': 'parameters'},
  {'Name': 'database_id',
   'Expression': '"dhfsh-gold" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=true]',
   'QueryGroup': 'parameters'},
  {'Name': 'storage_id',
   'Expression': '"sastddthglbfsh1rawweuuat" meta [IsParameterQuery=true, Type="Any", IsParameterQueryRequired=true]',
   'QueryGroup': 'parameters'}],
 'DAXCalculations': [{'Table': 'Mesure',
   'Measure': 'Nb of Clients who used the Wishlist',
   'Expression': ['', 'DISTINCTCOUNT(fact_wishlist[fky_client])', '']},
  {'Table': 'Mesure',
   'Measure': 'Wishlist usage frequency per client',
   'Expression': ['',
    'VAR distinct_days = ',
    "DISTINCTCOUNT( 'fact_wishlist'[lab_updatestatus] )",
    '',
    'VAR t =',
    '    SUMMARIZE(',
    '        fact_wishlist,',
    '        fact